In [1]:
import sys
import numpy as np
from math import ceil
import tensorflow as tf
from tensorflow.math import log
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB2
from tensorflow import zeros_like,reduce_sum,divide,exp,expand_dims
from tensorflow.keras.layers import Input, TimeDistributed,Flatten,Dense,Dropout,Reshape


sys.path.append('/kaggle/input/mlware-preprocessed')
from DataHelper import *

In [2]:
helpMe()

List of functions availabe to be used:


1.load_images(path,valid_split):
Takes in 2 parameters, returns 2 numpy arrays, corresponding to training and test set. 
    
    This function is used to load the images as arrays for training and validation.
    Format for loading images:
        -> Under path, there must be N folders, each of the folders having the n different splitted pieces of the image
        -> To generate the suitable format , use CreateDataset function(number 8 in Help) and cut->paste all the folders into a different directory.
    Example usage:
train,test=load_images("c:/images/puzzles",0.2)
    
    
2.generatePM(N,n,split):
Produces a (nxn) permutation matrix representing the ground truth for an unshuffled image. 
    
    N is the number of images for which permutation matrix has to be generated,
    n is the total number of pieces(square board)
    split is ratio between training and test set
    Example usage:
y_train,y_test=generatePM(train.shape[0]+test.shape[

In [3]:
x_train,x_test = load_images("/kaggle/input/mlware-preprocessed/Faces_train",valid_split=0.2)

In [4]:

y=PermutationMatrix("/kaggle/input/mlware-preprocessed/Preprocessed_faces.csv")
y_train,y_test=y[:ceil(y.shape[0]*0.8)],y[ceil(y.shape[0]*0.8):]


In [5]:
newx_train,newy_train=AugmentedData(x_train,y_train,6000,augments_per_image=15)

In [6]:
newx_test,newy_test=AugmentedData(x_test,y_test,400,augments_per_image=15)

In [7]:
class SinkhornLayer(tf.keras.layers.Layer):
    def __init__(self, epsilon=0.1, n_iters=10):
        super().__init__()
        self.epsilon = epsilon
        self.n_iters = n_iters

    def sinkhorn(self, x):
        # Sinkhorn iterations
        self.u = zeros_like(x[:, 0])
        self.v = zeros_like(x[0])
        for i in range(self.n_iters):
            self.u = -self.epsilon * (log(reduce_sum(exp(divide(x - expand_dims(self.v, axis=1), self.epsilon)), axis=2))) + self.u
            self.v = -self.epsilon * (log(reduce_sum(exp(divide(x - expand_dims(self.u, axis=2), self.epsilon)), axis=1))) + self.v

        # Compute the sinkhorn matrix
        sinkhorn_matrix = exp(divide(x - expand_dims(self.u, axis=2) - expand_dims(self.v, axis=1), self.epsilon))

        return sinkhorn_matrix
def create_siamese_network():
    # Define the input layer
    inputs = Input(shape=(36, 50, 50, 3))
    
    # Define the shared layers of the Siamese network using EffiecientNet architecture
    shared_layers = EfficientNetB2(
        input_shape=(50, 50, 3),
        include_top=False,
        weights="imagenet"
    )
    shared_layers.trainable = False

    # Apply the shared layers to each input image
    x = TimeDistributed(shared_layers)(inputs)
    x = Flatten()(x)

    # Add a fully connected layer to output a vector of length 1296
    fc1 = Dense(1024, activation='relu')(x)
    fc1 = Dropout(0.2)(fc1)  # Add dropout
    fc2 = Dense(512, activation='relu')(fc1)
    fc2 = Dropout(0.2)(fc2)  # Add dropout
    output = Dense(1296, activation='softmax')(fc2)
    
    # Reshape output to a 36x36 one-hot encoded matrix
    reshaped_output = Reshape((36, 36))(output)
    sinkhorn_output = SinkhornLayer()(reshaped_output)
    
    # Define the Siamese model with 36 inputs and one output
    model = Model(inputs=inputs, outputs= sinkhorn_output)
    

    return model

# Create the Siamese network with input shape (50, 50, 3) and outputting a 36x36 one-hot encoded matrix
siamese_network = create_siamese_network()
siamese_network.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.00001),metrics=["accuracy"])


31790344/31790344 [==============================] - 0s 0us/step


In [8]:
siamese_network.fit(newx_train,newy_train,validation_data=(newx_test,newy_test),epochs=500)

Epoch 1/500


2023-04-10 15:12:19.060056: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/time_distributed/efficientnetb2/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


190/190 [==============================] - 76s 283ms/step - loss: 3.5784 - accuracy: 0.0382 - val_loss: 3.5948 - val_accuracy: 0.0287
Epoch 2/500
190/190 [==============================] - 44s 229ms/step - loss: 3.5047 - accuracy: 0.0760 - val_loss: 3.6211 - val_accuracy: 0.0265
Epoch 3/500
190/190 [==============================] - 44s 233ms/step - loss: 3.3929 - accuracy: 0.1274 - val_loss: 3.6353 - val_accuracy: 0.0290
Epoch 4/500
190/190 [==============================] - 46s 243ms/step - loss: 3.2739 - accuracy: 0.1804 - val_loss: 3.6612 - val_accuracy: 0.0282
Epoch 5/500
190/190 [==============================] - 46s 243ms/step - loss: 3.1672 - accuracy: 0.2249 - val_loss: 3.6711 - val_accuracy: 0.0261
Epoch 6/500
190/190 [==============================] - 44s 233ms/step - loss: 3.0671 - accuracy: 0.2638 - val_loss: 3.6852 - val_accuracy: 0.0259
Epoch 7/500
190/190 [==============================] - 46s 243ms/step - loss: 2.9824 - accuracy: 0.2953 - val_loss: 3.6928 - val_accurac

In [9]:
siamese_network.save_weights("model_faces.h5")